In [1]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
ef = pd.read_csv('energy_flow.csv')
pitch_velo = pd.read_csv('metadata.csv')
pitch_velo = pitch_velo.drop(labels = ['filename_new', 'modelname_new', 'playing_level'], axis = 1)
ef = ef.dropna()
ef[:20]

,session_pitch,time,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,rear_knee_energy_generated,lead_knee_energy_transfer_stp,lead_knee_energy_transfer_jfp,lead_knee_energy_generated,rear_hip_energy_transfer_stp,rear_hip_energy_transfer_jfp,...,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
2,1031_2,0.0056,45.3350,20.1973,-8.0818,17.7538,6.0121,44.9851,0.1649,19.1142,...,1.3430,94.0265,-127.0096,65.5323,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
3,1031_2,0.0083,38.2959,19.3669,-8.7513,14.0252,4.4715,34.3861,0.0000,18.4929,...,1.5969,73.0565,-104.4922,57.6628,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
4,1031_2,0.0111,31.5861,18.5402,-9.5193,10.4432,3.0432,24.3333,0.0000,17.8197,...,1.4174,53.4393,-82.9323,50.1263,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
5,1031_2,0.0139,25.5195,17.7039,-10.4014,7.1676,1.7857,15.2168,0.0000,17.0917,...,0.8886,35.8775,-63.2113,43.2235,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
6,1031_2,0.0167,20.3575,16.8466,-11.4175,4.3307,0.7447,7.3494,0.0000,16.3050,...,0.1042,20.9140,-46.0584,37.2042,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
7,1031_2,0.0194,16.2879,15.9611,-12.5906,2.0265,-0.0497,0.9439,0.0000,15.4554,...,-0.8418,8.8945,-31.9950,32.2489,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
8,1031_2,0.0222,13.4109,15.0464,-13.9439,0.0000,-0.5844,-3.8984,0.0000,14.5387,...,-1.8623,-0.0501,-21.3023,28.4573,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
9,1031_2,0.0250,11.7354,14.1094,-15.4987,-0.8373,-0.8633,-7.1876,0.0000,13.5523,...,-2.8824,-5.9995,-14.0143,25.8448,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
10,1031_2,0.0278,11.1831,13.1642,-17.2711,-1.4393,-0.9046,-9.0342,-2.5615,12.4951,...,-3.8412,-9.2211,-9.9336,24.3473,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
11,1031_2,0.0306,11.6014,12.2316,-19.2705,-1.5833,-0.7381,-9.6299,-2.6995,11.3686,...,-4.6905,-10.1304,-8.6690,23.8330,0.625,1.1806,1.1917,1.3056,1.3389,1.3778


In [3]:
fp_br = ef[(ef['time']<= ef['BR_time']) & (ef['time']>= ef['MER_time'])]
fp_br.head()
##gets ball release frames from force plate data

,session_pitch,time,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,rear_knee_energy_generated,lead_knee_energy_transfer_stp,lead_knee_energy_transfer_jfp,lead_knee_energy_generated,rear_hip_energy_transfer_stp,rear_hip_energy_transfer_jfp,...,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
470,1031_2,1.3056,0.0,-494.8252,-282.9121,-226.4786,1009.0362,52.8333,-237.4349,44.1123,...,-2454.0339,-1238.2278,361.3429,-595.8149,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
471,1031_2,1.3083,0.0,-482.9549,-329.3855,-97.9962,1027.1194,218.5767,-186.1220,37.6231,...,-1587.4968,-1368.8935,432.5075,-628.9961,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
472,1031_2,1.3111,0.0,-463.0140,-378.9882,0.0000,1030.8112,413.9270,-125.3271,28.9622,...,-692.8811,-1441.7761,514.5392,-674.9362,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
473,1031_2,1.3139,0.0,-429.3051,-428.5289,0.0000,1025.2682,628.3532,-59.4795,15.0161,...,190.2121,-1461.6817,600.8485,-727.2714,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
474,1031_2,1.3167,0.0,-375.5942,-474.2034,0.0000,1015.0907,850.4686,0.0000,-7.6329,...,1019.9141,-1438.6833,682.3580,-776.8301,0.625,1.1806,1.1917,1.3056,1.3389,1.3778


In [4]:
fp_br.groupby("session_pitch").sum()
br = fp_br.drop(labels = ['time', 'pkh_time', 'fp_10_time',
                                'fp_100_time', 'MER_time','BR_time','MIR_time'], axis = 1)
br.head()

,session_pitch,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,rear_knee_energy_generated,lead_knee_energy_transfer_stp,lead_knee_energy_transfer_jfp,lead_knee_energy_generated,rear_hip_energy_transfer_stp,rear_hip_energy_transfer_jfp,rear_hip_energy_generated,...,pelvis_leadhip_seg_pwr,pelvis_rearhip_seg_pwr,pelvis_thorax_seg_pwr,rear_shank_dist_seg_pwr,rear_shank_prox_seg_pwr,thorax_dist_glove_seg_pwr,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr
470,1031_2,0.0,-494.8252,-282.9121,-226.4786,1009.0362,52.8333,-237.4349,44.1123,168.0203,...,169.1340,-193.3226,-172.6396,312.9028,-114.9925,56.8914,-2454.0339,-1238.2278,361.3429,-595.8149
471,1031_2,0.0,-482.9549,-329.3855,-97.9962,1027.1194,218.5767,-186.1220,37.6231,284.0086,...,300.0833,-148.4989,-187.6054,299.6106,-105.9599,19.7775,-1587.4968,-1368.8935,432.5075,-628.9961
472,1031_2,0.0,-463.0140,-378.9882,0.0000,1030.8112,413.9270,-125.3271,28.9622,418.1743,...,437.5925,-96.3649,-249.4881,295.9481,-103.3969,-6.4899,-692.8811,-1441.7761,514.5392,-674.9362
473,1031_2,0.0,-429.3051,-428.5289,0.0000,1025.2682,628.3532,-59.4795,15.0161,556.3852,...,578.0525,-44.4633,-347.9156,298.7425,-107.8456,-24.8248,190.2121,-1461.6817,600.8485,-727.2714
474,1031_2,0.0,-375.5942,-474.2034,0.0000,1015.0907,850.4686,0.0000,-7.6329,680.1171,...,717.6206,-2.2409,-466.6335,302.6268,-118.7508,-37.8482,1019.9141,-1438.6833,682.3580,-776.8301


In [23]:
bins = (69,71,73,75,77,79,81,83,85,87,89,91,93,95)
pitch_velo['mph_bin'] = pd.cut(pitch_velo['pitch_speed_mph'], bins = bins)
pitch_velo[:10]

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,pitch_speed_mph,mph_bin
0,750,1031,1031_2,110.6784,1.9304,21.40,90.4,"(89, 91]"
1,750,1031,1031_3,110.6784,1.9304,21.40,90.4,"(89, 91]"
2,774,1097,1097_1,101.1528,1.9304,19.41,77.6,"(77, 79]"
3,774,1097,1097_2,101.1528,1.9304,19.41,77.0,"(75, 77]"
4,774,1097,1097_3,101.1528,1.9304,19.41,76.1,"(75, 77]"
5,653,1170,1170_1,82.1016,1.9304,20.52,80.9,"(79, 81]"
6,653,1170,1170_2,82.1016,1.9304,20.52,82.6,"(81, 83]"
7,738,1250,1250_1,78.4728,1.7526,21.49,88.1,"(87, 89]"
8,738,1250,1250_3,78.4728,1.7526,21.49,88.6,"(87, 89]"
9,789,1313,1313_1,84.3696,1.8288,19.02,85.8,"(85, 87]"


In [24]:
label_bin = LabelEncoder()

In [25]:
pitch_velo['mph_bin'] = label_bin.fit_transform(pitch_velo['mph_bin'])
pitch_velo['mph_bin'].value_counts()

8     73
9     64
7     57
10    50
6     48
5     44
4     22
11    22
3     10
2      8
12     6
1      5
0      2
Name: mph_bin, dtype: int64

In [26]:
df = pd.merge(pitch_velo, br, how='inner', left_on='session_pitch', right_on='session_pitch')
df.head()

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,pitch_speed_mph,mph_bin,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,...,pelvis_leadhip_seg_pwr,pelvis_rearhip_seg_pwr,pelvis_thorax_seg_pwr,rear_shank_dist_seg_pwr,rear_shank_prox_seg_pwr,thorax_dist_glove_seg_pwr,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr
0,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-494.8252,...,169.1340,-193.3226,-172.6396,312.9028,-114.9925,56.8914,-2454.0339,-1238.2278,361.3429,-595.8149
1,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-482.9549,...,300.0833,-148.4989,-187.6054,299.6106,-105.9599,19.7775,-1587.4968,-1368.8935,432.5075,-628.9961
2,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-463.0140,...,437.5925,-96.3649,-249.4881,295.9481,-103.3969,-6.4899,-692.8811,-1441.7761,514.5392,-674.9362
3,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-429.3051,...,578.0525,-44.4633,-347.9156,298.7425,-107.8456,-24.8248,190.2121,-1461.6817,600.8485,-727.2714
4,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-375.5942,...,717.6206,-2.2409,-466.6335,302.6268,-118.7508,-37.8482,1019.9141,-1438.6833,682.3580,-776.8301


In [27]:
df.head()

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,pitch_speed_mph,mph_bin,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,...,pelvis_leadhip_seg_pwr,pelvis_rearhip_seg_pwr,pelvis_thorax_seg_pwr,rear_shank_dist_seg_pwr,rear_shank_prox_seg_pwr,thorax_dist_glove_seg_pwr,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr
0,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-494.8252,...,169.1340,-193.3226,-172.6396,312.9028,-114.9925,56.8914,-2454.0339,-1238.2278,361.3429,-595.8149
1,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-482.9549,...,300.0833,-148.4989,-187.6054,299.6106,-105.9599,19.7775,-1587.4968,-1368.8935,432.5075,-628.9961
2,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-463.0140,...,437.5925,-96.3649,-249.4881,295.9481,-103.3969,-6.4899,-692.8811,-1441.7761,514.5392,-674.9362
3,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-429.3051,...,578.0525,-44.4633,-347.9156,298.7425,-107.8456,-24.8248,190.2121,-1461.6817,600.8485,-727.2714
4,750,1031,1031_2,110.6784,1.9304,21.4,90.4,10,0.0,-375.5942,...,717.6206,-2.2409,-466.6335,302.6268,-118.7508,-37.8482,1019.9141,-1438.6833,682.3580,-776.8301


In [28]:
#creating test and training splits to train and test the model
X = df.drop(labels = ['mph_bin', 'pitch_speed_mph', 'user', 
                      'age_yrs', 'session', 'session_pitch'], axis = 1)
y = df['mph_bin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 12)

In [29]:
#scale values
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
predict_rfc = rfc.predict(X_test)

In [35]:
print(classification_report(y_test, predict_rfc))
print(confusion_matrix(y_test, predict_rfc))
##67, 73, 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      0.80      0.89        10
           2       0.90      1.00      0.95        18
           3       1.00      1.00      1.00        12
           4       1.00      1.00      1.00        56
           5       0.99      0.97      0.98       117
           6       0.91      0.98      0.95        88
           7       0.97      0.96      0.97       163
           8       0.98      0.95      0.97       192
           9       0.96      0.94      0.95       159
          10       0.91      0.98      0.94       117
          11       0.94      0.91      0.92        53
          12       0.96      0.92      0.94        26

    accuracy                           0.96      1019
   macro avg       0.96      0.96      0.96      1019
weighted avg       0.96      0.96      0.96      1019

[[  8   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   8   2   0   0   

In [32]:
clf = svm.SVC()
clf.fit(X_train, y_train)
predict_clf = clf.predict(X_test)

In [33]:
print(classification_report(y_test, predict_clf))
print(confusion_matrix(y_test, predict_clf))
#consistently 50%

              precision    recall  f1-score   support

           0       0.73      1.00      0.84         8
           1       1.00      0.30      0.46        10
           2       0.68      0.72      0.70        18
           3       0.67      0.83      0.74        12
           4       0.88      0.66      0.76        56
           5       0.73      0.85      0.79       117
           6       0.77      0.85      0.81        88
           7       0.89      0.80      0.85       163
           8       0.86      0.92      0.89       192
           9       0.92      0.83      0.87       159
          10       0.71      0.89      0.79       117
          11       0.91      0.57      0.70        53
          12       0.86      0.69      0.77        26

    accuracy                           0.82      1019
   macro avg       0.82      0.76      0.77      1019
weighted avg       0.83      0.82      0.82      1019

[[  8   0   0   0   0   0   0   0   0   0   0   0   0]
 [  3   3   4   0   0   

In [38]:
pitch_velo['pitch_speed_mph'].min()

69.5